In [2]:
# Reference: https://stackoverflow.com/questions/34199233/how-to-prevent-tensorflow-from-allocating-the-totality-of-a-gpu-memory
import tensorflow as tf
opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
conf = tf.ConfigProto(gpu_options=opts)
tf.enable_eager_execution(config=conf)

In [3]:
# Reference: https://stackoverflow.com/questions/42064690/using-pre-trained-word2vec-with-lstm-for-word-generation
# https://rare-technologies.com/word2vec-tutorial/
# Local: pip install gensim
# GCP: conda install gensim (pip instal didn't work)
import random
import sys
import os
import re
import numpy as np
# import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding
import gensim

In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
# max_sentence_len = 40
# documents = ["Human machine interface for lab abc computer applications . ",
#              "A survey of user opinion of computer system response time , ",
#              "The EPS user interface management system",
#              "System and human system engineering testing of EPS",
#              "Relation of user perceived response time to error measurement",
#              "The generation of random binary unordered trees",
#              "The intersection graph of paths in trees",
#              "Graph minors IV Widths of trees and well quasi ordering",
#              "Graph minors A survey . maybe"]
# sentences = [[word for word in doc.lower().split()[:max_sentence_len]] for doc in documents]
# sentences
# word_model = gensim.models.Word2Vec(sentences, size=100, min_count=1, window=5, iter=100)
# pretrained_weights = word_model.wv.syn0
# vocab_size, emdedding_size = pretrained_weights.shape
# print('Result embedding shape:', pretrained_weights.shape)

# def word2idx(word):
#       return word_model.wv.vocab[word].index
# def idx2word(idx):
#       return word_model.wv.index2word[idx]

# print('1. ', word2idx("thirteen"))
# print('2. ', word_model.wv.vocab["thirteen"].index)

In [6]:
dir_split = f"../1.DataPreparationResults/obama"
file_train = open(f"{dir_split}/train.txt", 'r').read()
file_val = open(f"{dir_split}/val.txt", 'r').read()
file_test = open(f"{dir_split}/test.txt", 'r').read()

# Windows ONLY
# dir_split = r"D:\UCBerkeley\CourseWork\201909\W266\GitHub\FinalProject-Collab\1.DataPreparationResults\obama"
# file_train = open(f"{dir_split}\\train.txt", 'r').read()
# file_val = open(f"{dir_split}\\val.txt", 'r').read()
# file_test = open(f"{dir_split}\\test.txt", 'r').read()

In [7]:
x_len = 30
x_step = 1

In [8]:
# os.path.exists(u"gs://w266-final-project/GoogleNews-vectors-negative300.bin")
os.path.exists("../../test/GoogleNews-vectors-negative300.bin")

True

In [9]:
# Google pre-trained word2vec model
# Reference: https://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/
# Reference: https://code.google.com/archive/p/word2vec/
# google_word_model = gensim.models.KeyedVectors.load_word2vec_format(r"D:\UCBerkeley\CourseWork\201909\W266\GitHub\FinalProject-Collab\GoogleNews-vectors-negative300.bin", binary=True)



google_word_model = gensim.models.KeyedVectors.load_word2vec_format('../../test/GoogleNews-vectors-negative300.bin', binary=True)
pretrained_weights = google_word_model.wv.vectors
print(type(pretrained_weights))
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)

<class 'numpy.ndarray'>
Result embedding shape: (3000000, 300)


/home/tennisonyu/anaconda3/envs/w266-lstm/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


In [10]:
def word2idx(word):
    return google_word_model.wv.vocab[word].index
# def word2idx(word):
#     if word in google_word_model.wv.vocab:
#         return google_word_model.wv.vocab[word].index
#     # Capitalization matters in Google trained word2vec
#     # "united" is not equal to "United"
#     elif type(word[0]) == str:
#         word = word[0].lower() + word[1:]
#         if word in google_word_model.wv.vocab:
#             return google_word_model.wv.vocab[word].index
#     else:
#         return -1
def idx2word(idx):
    return google_word_model.wv.index2word[idx]

vocab = google_word_model.wv.vocab
# Confirm that word_model works
# print(word_model.wv.vocab["doubts"].index)

/home/tennisonyu/anaconda3/envs/w266-lstm/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [11]:
# The following output is too large, which will make the notebook render super slowly
# for word in vocab:
#     if '_' in word:
#         print(word)

* Google pretrained words don't include 3 common stop words: 'a', 'and', 'of', and 'to'. Interestingly, it includes 'A', 'And', 'Of', and 'To'.  
* Google pretrained model assumes all digits are replaced by #.  
* Google combines common phrases with '_' (TO BE IMPLEMENTED)

In [12]:
def google_preprocess(file):
    file2 = re.sub('\d', '#', file)
    file2 = re.sub(' a ', ' A ', file2)
    file2 = re.sub(' and ', ' And ', file2)
    file2 = re.sub(' of ', ' Of ', file2)
    file2 = re.sub(' to ', ' To ', file2)
    # Add spaces around <speech_sep>
    # Create a set of all words in file.txt but remove <speech_sep>
    unique_words = set(file2.replace("<speech_sep>", " <speech_sep> ").split())
    unique_words.remove("<speech_sep>")
    return file2, unique_words

file_train_google, unique_words_train = google_preprocess(file_train)
file_val_google, unique_words_val = google_preprocess(file_val)
file_test_google, unique_words_test = google_preprocess(file_test)

unique_words_all = unique_words_train.union(unique_words_val.union(unique_words_test))
print("total number of unique words: ",len(unique_words_all))

total number of unique words:  10774


In [13]:
def file_to_sentences(file):
    sentences = []
    sentences2 = []
    next_words = []
    list_words = []
    
    for speech in file.split("<speech_sep>"):
        list_words = speech.split()
        # I noticed the last speech has zero word 
        # because <speech_sep> is the last character
        if len(list_words) == 0:
            break
        
        # each row should have x_len + 1 (both input and target)
        for i in range(0,len(list_words)-x_len-1, x_step):
            sentences2 = [word for word in list_words[i: i + x_len + 1]]
            sentences.append(sentences2)
            
    return sentences

# train_sentences = file_to_sentences(file_train)
train_sentences = file_to_sentences(file_train_google)

# print('sentences rows = ', len(train_sentences))
# print('sentences columns = ', len(train_sentences[0]))
# print(train_sentences[0])

In [14]:
# Train word2vec model
# window: The maximum distance between the current and predicted word within a sentence
# The code below will not account for words in val/test.txt that do not exist in train.txt
# Reference: https://radimrehurek.com/gensim/models/doc2vec.html
# word_model = gensim.models.Word2Vec(train_sentences, size=100, min_count=1, window=x_len+1, iter=5)
# pretrained_weights = word_model.wv.vectors
# print(type(pretrained_weights))

# def word2idx(word):
#     return word_model.wv.vocab[word].index

# def idx2word(idx):
#     return word_model.wv.index2word[idx]

# vocab = google_word_model.wv.vocab
# Confirm that word_model works
# print(word_model.wv.vocab["doubts"].index)

In [15]:
pretrained_weights_mini = []
vocab_mini_lst = []
vocab_mini_dict = dict()
i = 0
for word in unique_words_all:
    if word in google_word_model.wv.vocab:
        i += 1
        pretrained_weights_mini.append(pretrained_weights[google_word_model.wv.vocab[word].index][:100])
        vocab_mini_lst.append(word)
        vocab_mini_dict[word] = i
        
pretrained_weights_mini = np.array(pretrained_weights_mini)
print(pretrained_weights_mini.shape)
vocab_size, emdedding_size = pretrained_weights.shape
print(len(vocab_mini_lst))

def word2idx(word):
    return vocab_mini_dict[word]
    # The code below works but is too slow when executing sentences_to_2darray
    # vocab_mini_lst.index(word)
    
# def word2idx(word):
#     if word in google_word_model.wv.vocab:
#         return google_word_model.wv.vocab[word].index
#     # Capitalization matters in Google trained word2vec
#     # "united" is not equal to "United"
#     elif type(word[0]) == str:
#         word = word[0].lower() + word[1:]
#         if word in google_word_model.wv.vocab:
#             return google_word_model.wv.vocab[word].index
#     else:
#         return -1
def idx2word(idx):
    return vocab_mini_lst[idx]

(10679, 100)
10679


/home/tennisonyu/anaconda3/envs/w266-lstm/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/home/tennisonyu/anaconda3/envs/w266-lstm/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [16]:
pretrained_weights[1][:200].shape

(200,)

In [17]:
idx2word(1)

'standardized'

In [18]:
'in' in unique_words_all

True

In [19]:
google_word_model.wv.vocab['To']

/home/tennisonyu/anaconda3/envs/w266-lstm/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [20]:
def sentences_to_2darray(sentences):
    
    missing_words = set()
    
    x = np.zeros([len(sentences), x_len], dtype=np.int32)
    y = np.zeros([len(sentences)], dtype=np.int32)
    for i, sentence in enumerate(sentences):
        for t, word in enumerate(sentence[:-1]):
            if word in vocab:
                x[i, t] = word2idx(word)
#             elif type(word[0]) == str:
#                 word = word[0].lower() + word[1:]
#                 if word in vocab:
#                     x[i, t] = word2idx(word)
            else:
                x[i, t] = 0
                missing_words.add(word)
        if sentence[-1] in vocab:
            y[i] = word2idx(sentence[-1])
        else:
            y[i] = 0
    print(missing_words) 
        
    return x, y

train_X, train_Y = sentences_to_2darray(train_sentences)
print('train_X shape:', train_X.shape)
print('train_Y shape:', train_Y.shape)

{'Iyou', ',', 'NASbut', 'travelled', 'NASAnd', 'NASfrom', 'SNAbecause', 'DePayne', 'eacute', 'nothe', 'Tywanza', 'Ithe', 'iuml', 'NASour', 'grey', 'overspilled', 'copouts', 'REa', 'NASand', '#st', 'Clementa', 'and', 'UCThank', '.', '#th', 'travelling', 'Ritterby', 'doughnut', 'CASo', 'Iwhen', 'NASWe'}
train_X shape: (79954, 30)
train_Y shape: (79954,)


In [21]:
# val_X, val_Y = sentences_to_2darray(file_to_sentences(file_val))
val_X, val_Y = sentences_to_2darray(file_to_sentences(file_val_google))
print(val_X.shape)
print(val_Y.shape)

{'tranquillity', 'initiativeto', ',', 'IFewer', 'salaamu', 'ISIWe', 'Cannot', 'iuml', 'whyI', 'Dbehind', 'MyRA', 'COBRand', 'Tput', 'Nelba', '#D', 'Estiven', 'oughtness', 'spires', 'Jobses', 'inreconcilable', 'cuentapropistas', 'worshippers', 'frac##', '#rd', 'MyRIt', 'NASand', 'UPand', 'futurode', 'BIn', '#st', 'Cachita', 'Sheoma', '.', 'Cultivo', '#th', 'Borlaugs', 'AIDAnd', 'inventa', 'VAnd', 'sayyes', 'NOAand', 'Iwhen', 'pretence'}
(68110, 30)
(68110,)


In [22]:
mirrored_strategy = tf.distribute.MirroredStrategy()

In [27]:
# Monitor the GPU memory usage: nvidia-smi

with mirrored_strategy.scope():
    model = keras.Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=150, weights=[pretrained_weights_mini]))
    model.add(LSTM(emdedding_size))
    model.add(Dropout(0.2))
    # model.add(LSTM(emdedding_size))
    # model.add(Dropout(0.2))
    model.add(Dense(vocab_size, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')


ResourceExhaustedError: OOM when allocating tensor with shape[3000000,50] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Mul] name: embedding_4/embeddings/Initializer/random_uniform/mul/

In [ ]:
model.fit(train_X, train_Y, epochs=2, batch_size=1)

In [7]:
if os.path.isfile('GoTweights'):
    model.load_weights('GoTweights')

def sample(a, temperature=1.0):
    # helper function to randomly select a word based on probability distribution
#     a = np.log(a) / temperature
#     a = np.exp(a) / np.sum(np.exp(a))
#     return np.argmax(np.random.multinomial(1, a, 1))
    
    # Reference: https://github.com/llSourcell/How-to-Generate-Music-Demo/issues/4
    a = np.log(a) / temperature 
    dist = np.exp(a)/np.sum(np.exp(a)) 
    choices = range(len(a)) 
    return np.random.choice(choices, p=dist)

# train the model, output generated text after each iteration
for iteration in range(1, 5):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(train_X, train_Y, batch_size=1280, epochs=2, validation_data=(val_X,val_Y))
    model.save_weights('GoTweights',overwrite=True)

    # Select a speech from the test file
    # randint(a,b) selects from all integers between a and b (inclusive)
    # The last speech has zero word, so instead of -1 use -2
    gen_speech_index = random.randint(0, len(file_test.split("<speech_sep>"))-2)
    # A list of words in the speech
    list_words = file_test.split("<speech_sep>")[gen_speech_index].split()
    # Select a starting point for the context
    start_index = random.randint(0, len(list_words) - x_len - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)
        generated = ''
        sentence = list_words[start_index: start_index + x_len]
        generated += ' '.join(sentence)
        print('----- Generating with seed: "' , sentence , '"')
        print()
        sys.stdout.write(generated)
        print()

        for i in range(50):
            x = np.zeros((1, x_len, len(word_train)))
            for t, word in enumerate(sentence):
                if word in word_train:
                    x[0, t, word_indices[word]] = 1.

            # model.predict(x, verbose=0).shape = (1, x_len)
            # model.predict(x, verbose=0)[0].shape = (x_len, )
            preds = model.predict(x, verbose=0)[0]            
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]
            generated += next_word
            del sentence[0]
            sentence.append(next_word)
            sys.stdout.write(' ')
            sys.stdout.write(next_word)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Train on 80001 samples, validate on 83061 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/2
80001/80001 [==============================] - 294s 4ms/sample - loss: 6.6451 - val_loss: 5.9541
Epoch 2/2
80001/80001 [==============================] - 285s 4ms/sample - loss: 6.4338 - val_loss: 5.8888

----- diversity: 0.2
----- Generating with seed: " ['rocket', 'attacks', 'from', 'Gaza', ',', 'and', 'we', 'have', 'stood', 'up', 'for', 'Israel', 's', 'right', 'to', 'defend', 'itself', '.', 'And', 'that', 'is', 'why', 'Israel', 'has', 'a', 'right', 'to', 'expect', 'Hamas', 'to'] "

rocket attacks from Gaza , and we have stood up for Israel s right to defend itself . And that is why Israel has a right to expect Hamas to
 . . , that . . , , that . . . , , to , that the , . , . . . . . to . . , the . the , . , . the the , . . , . . . , . , .

----- diversity: 0.5
----- Ge

 that just but to But on escape are was class shut family all need and their chance . housing an earth for hospital I families all but at insurance that Air the use , she jobs and insurance have must argument at Yet we with a the destiny who plainly

----- diversity: 1.2
----- Generating with seed: " ['.', 'Imagine', 'imagine', 'for', 'a', 'moment', ',', 'here', 'was', 'a', 'young', 'girl', 'who', 'was', 'just', 'becoming', 'aware', 'of', 'our', 'democracy', 'just', 'beginning', 'to', 'understand', 'the', 'obligations', 'of', 'citizenship', 'just', 'starting'] "

. Imagine imagine for a moment , here was a young girl who was just becoming aware of our democracy just beginning to understand the obligations of citizenship just starting
 to rebuild actually Scranton their steal while percent to freedom the himself to can stems got could between even makes now proposal cannot push the stop United this to feed flag is economy portion we that I loopholes crisis many made together to patrioti

In [10]:
model.predict(x, verbose=0)[0].shape

(6508,)